In [1]:
# Taken from main06.cc Pythia example. A study of event properties of LEP1 events

from numpythia import Pythia, hepmc_write, hepmc_read
from numpythia import STATUS, HAS_END_VERTEX, ABS_PDG_ID

from pyjet import cluster, DTYPE_PTEPM, DTYPE_EP
from pyjet.testdata import get_event

import numpy as np
import sys

from pyjet import cluster
from pyjet import ClusterSequenceArea, JetDefinition, ClusterSequence

from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

In [2]:
# Pythia generator settings. Process selection e+e- collision.
# WeakSingleBoson:ffbar2gmZ : Produce Z bosons and interference with photon
# Consider only Z (PDGID=23) decays to light quarks

params = {"Beams:idA" : "11", "Beams:idB" : "-11","Beams:eCM": 91.1876, "WeakSingleBoson:ffbar2gmZ" : "on", "23:onMode": "off", "23:onIfAny": "1 2 3 4 5", "PDF:lepton": "off"}
pythia = Pythia(params =params, random_state=1)

# Consider only final-state particles
selection = ((STATUS == 1) & ~HAS_END_VERTEX)

In [3]:
# Array of pseudo-jets or particles to cluster
particles = []
array = []

# Begin event loop. Generate events
for event in pythia(events=1):
    array = event.all(selection)

In [7]:
particles = array[array["E"] > 2.0]

In [8]:
print(len(particles))

13


In [9]:
len(array)

35

In [ ]:
# Print out each momentum (starting from 0 to enable gnuplot to easily draw a line)
with open('ee_particles.dat', 'w') as f:
    for p in particles:
        f.write("%s %s %s %s\n" %( p['E'], p['px'], p['py'], p['pz']))


In [10]:
# Print out each momentum (starting from 0 to enable gnuplot to easily draw a line)
with open('out.particles', 'w') as f:
    for p in particles:
        f.write("0 0 0 0\n")
        f.write("%s %s %s %s\n\n\n" %( p['px'], p['py'], p['pz'], p['E']))


In [11]:
# Cluster particles into jets
# First generate a whole "clustering sequence" with the e+e- kt algorithm

jetdef = JetDefinition('ee_kt')
sequence = ClusterSequence(particles, jetdef, ep=True)

In [12]:
# Then select the jets that come out of clustering 
jets = sequence.exclusive_jets(3)

In [13]:
with open('out.jetconst', 'w') as f:
    for c in range(len(jets[0].constituents_array(ep=True))):
        f.write("0 0 0 0\n")
        f.write("%s %s %s %s\n\n\n" %(jets[0].constituents_array(ep=True)[c]['px'], jets[0].constituents_array(ep=True)[c]['py'], jets[0].constituents_array(ep=True)[c]['pz'], jets[0].constituents_array(ep=True)[c]['E']))

In [14]:
# Print out the jets to a file
with open('out.ktjets', 'w') as f:
    for jet in jets:
        f.write("0 0 0 0\n")
        f.write("%s %s %s %s\n\n\n" %(jet.px, jet.py, jet.pz, jet.e))

In [15]:
# Create qubo
from pyqubo import Spin, Array

# Create an array os spin variables
n_part = len(particles) 
s = Array.create('s', shape=n_part, vartype='BINARY')

# Create an array of zeroes for qubo matrix coefficients
coeff = [[0] * n_part for _ in range(n_part)] 
theta = np.pi/4.

for i in range(0,n_part):
    for j in range(0,n_part):
        coeff[i][j] = (particles[i]['px']*particles[j]['px'] + particles[i]['py']*particles[j]['py'] + particles[i]['pz']*particles[j]['pz'] - particles[i]['E']*particles[j]['E'] * np.cos(theta))/(1 - np.cos(theta))

# Construct Hamiltonian
H = sum([ -1.0*coeff[i][j]*s[i]*s[j] for i in range(0,n_part) for j in range(0,n_part)])

# Compile model using pyqubo

model = H.compile()
qubo, offset = model.to_qubo()

In [56]:
# Submit qubo to D-Wave

sampler = EmbeddingComposite(DWaveSampler(solver={'qpu':True}))
response = sampler.sample_qubo(qubo)

In [57]:
response

SampleSet(rec.array([([0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1], -106.75159779, 1, 0.84615385)],
          dtype=[('sample', 'i1', (13,)), ('energy', '<f8'), ('num_occurrences', '<i8'), ('chain_break_fraction', '<f8')]), ['s[0]', 's[10]', 's[11]', 's[12]', 's[1]', 's[2]', 's[3]', 's[4]', 's[5]', 's[6]', 's[7]', 's[8]', 's[9]'], {'timing': {'qpu_sampling_time': 239, 'qpu_anneal_time_per_sample': 20, 'qpu_readout_time_per_sample': 198, 'qpu_access_time': 11015, 'qpu_access_overhead_time': 1275, 'qpu_programming_time': 10776, 'qpu_delay_time_per_sample': 21, 'total_post_processing_time': 456, 'post_processing_overhead_time': 456, 'total_real_time': 11015, 'run_time_chip': 239, 'anneal_time_per_run': 20, 'readout_time_per_run': 198}, 'problem_id': '5b3e9d30-ca53-42e7-923d-7ee18e57018e'}, 'BINARY')

In [58]:
for datum in response.data(['sample', 'energy']):
    print(datum.sample, datum.energy)

{'s[0]': 0, 's[10]': 1, 's[11]': 0, 's[12]': 1, 's[1]': 0, 's[2]': 0, 's[3]': 0, 's[4]': 0, 's[5]': 0, 's[6]': 0, 's[7]': 1, 's[8]': 0, 's[9]': 1} -106.75159779401137


In [59]:
solution = {'s[0]': 0, 's[10]': 1, 's[11]': 0, 's[12]': 1, 's[1]': 0, 's[2]': 0, 's[3]': 0, 's[4]': 0, 's[5]': 0, 's[6]': 0, 's[7]': 1, 's[8]': 0, 's[9]': 1}

In [60]:
with open('out.particlesqujet', 'w') as f:
    for i in range(len(particles)):
        key = 's[%s]' %i
        if(solution[key]==1):
            f.write("0 0 0 0\n")
            f.write("%s %s %s %s\n\n\n" %( particles[i]['px'], particles[i]['py'], particles[i]['pz'], particles[i]['E']))

In [62]:
# Print out each momentum (starting from 0 to enable gnuplot to easily draw a line)
with open('ee_particles_qpu.dat', 'w') as f:
    for i in range(len(particles)):
        key = 's[%s]' %i
        f.write("%s %s %s %s %s\n" %( particles[i]['E'],particles[i]['px'], particles[i]['py'], particles[i]['pz'], solution[key]))


In [ ]:
import pyqubo
decoded_solution, broken, energy = model.decode_solution(solution, vartype='BINARY')

In [19]:
import dimod 
solution_dimod = dimod.ExactSolver().sample_qubo(qubo)

In [20]:
solution_dimod.lowest()

SampleSet(rec.array([([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1], -1317.94186794, 1)],
          dtype=[('sample', 'i1', (13,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), ['s[0]', 's[10]', 's[11]', 's[12]', 's[1]', 's[2]', 's[3]', 's[4]', 's[5]', 's[6]', 's[7]', 's[8]', 's[9]'], {}, 'BINARY')

In [24]:
for datum in solution_dimod.data(['sample', 'energy']):
    if (datum.energy < -1000):
        print(datum.sample, datum.energy)

{'s[0]': 0, 's[10]': 1, 's[11]': 1, 's[12]': 1, 's[1]': 0, 's[2]': 0, 's[3]': 0, 's[4]': 0, 's[5]': 1, 's[6]': 1, 's[7]': 1, 's[8]': 1, 's[9]': 1} -1317.9418679383082
{'s[0]': 1, 's[10]': 0, 's[11]': 0, 's[12]': 0, 's[1]': 1, 's[2]': 1, 's[3]': 1, 's[4]': 1, 's[5]': 0, 's[6]': 0, 's[7]': 0, 's[8]': 0, 's[9]': 0} -1301.7697919391849
{'s[0]': 0, 's[10]': 0, 's[11]': 1, 's[12]': 1, 's[1]': 0, 's[2]': 0, 's[3]': 0, 's[4]': 0, 's[5]': 1, 's[6]': 1, 's[7]': 1, 's[8]': 1, 's[9]': 1} -1179.316088893212
{'s[0]': 1, 's[10]': 0, 's[11]': 0, 's[12]': 0, 's[1]': 1, 's[2]': 1, 's[3]': 1, 's[4]': 0, 's[5]': 0, 's[6]': 0, 's[7]': 0, 's[8]': 0, 's[9]': 0} -1177.204767718082
{'s[0]': 0, 's[10]': 1, 's[11]': 1, 's[12]': 0, 's[1]': 0, 's[2]': 0, 's[3]': 0, 's[4]': 0, 's[5]': 1, 's[6]': 1, 's[7]': 1, 's[8]': 1, 's[9]': 1} -1175.077215508937
{'s[0]': 1, 's[10]': 0, 's[11]': 0, 's[12]': 0, 's[1]': 1, 's[2]': 1, 's[3]': 0, 's[4]': 1, 's[5]': 0, 's[6]': 0, 's[7]': 0, 's[8]': 0, 's[9]': 0} -1159.1314485219223
{

In [25]:
dimod_list = {'s[0]': 0, 's[10]': 1, 's[11]': 1, 's[12]': 1, 's[1]': 0, 's[2]': 0, 's[3]': 0, 's[4]': 0, 's[5]': 1, 's[6]': 1, 's[7]': 1, 's[8]': 1, 's[9]': 1}

In [27]:
with open('out.particlesdimod', 'w') as f:
    for i in range(0,len(particles)):
        key = 's[%s]' %i
        if(dimod_list[key]==1):
            f.write("0 0 0 0\n")
            f.write("%s %s %s %s\n\n\n" %( particles[i]['px'], particles[i]['py'], particles[i]['pz'], particles[i]['E']))

In [63]:
# Print out each momentum (starting from 0 to enable gnuplot to easily draw a line)
with open('ee_particles_dimod.dat', 'w') as f:
    for i in range(len(particles)):
        key = 's[%s]' %i
        f.write("%s %s %s %s %s\n" %( particles[i]['E'],particles[i]['px'], particles[i]['py'], particles[i]['pz'], dimod_list[key]))
